In [4]:
!pip install av
! wget https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
# Download HMDB51 data and splits from serre lab website
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar

     |████████████████████████████████| 37.2MB 78kB/s 
--2021-07-12 13:06:11--  https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3102 (3.0K) [text/plain]
Saving to: ‘transforms.py.1’

transforms.py.1     100%[===================>]   3.03K  --.-KB/s    in 0s      

2021-07-12 13:06:11 (48.1 MB/s) - ‘transforms.py.1’ saved [3102/3102]

--2021-07-12 13:06:11--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Resolving serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)... 128.148.254.114
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:80... connected.
HTTP request sent

In [5]:
pip install git+https://github.com/Atze00/MoViNet-pytorch.git

  Cloning https://github.com/Atze00/MoViNet-pytorch.git (to revision createpackage) to /tmp/pip-req-build-_wnzohv6
  Running command git clone -q https://github.com/Atze00/MoViNet-pytorch.git /tmp/pip-req-build-_wnzohv6
  Running command git checkout -b createpackage --track origin/createpackage
  Switched to a new branch 'createpackage'
  Branch 'createpackage' set up to track remote branch 'createpackage' from 'origin'.
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 645kB 35.7MB/s 
  Created wheel for MoViNet-pytorch: filename=MoViNet_pytorch-0.0.1-cp37-none-any.whl size=10502 sha256=0d9475c5f59a14483e710580c17a11f895fc229fae7a0173ef8291d914926286
  Stored in directory: /tmp/pip-ephem-wheel-cache-tjetku_i/wheels/ea/71/a5/da90c07e42afdd3d864703664a0956da57ce28de5eff1b2dea
  Created wheel for fvcore: filename=fvcore-0.1.5.post20210630-cp37-none-any.whl size=60611 sha256=26d4a7cc09539de41a242c1843244fbeaaa4889bf66f271f1a840fde21f3b7c7
  Sto

In [6]:
# Extract and organize video data..
! mkdir -p video_data test_train_splits
! unrar e test_train_splits.rar test_train_splits
! rm test_train_splits.rar
! unrar e hmdb51_org.rar 
! rm hmdb51_org.rar
! mv *.rar video_data
import os
for files in os.listdir('video_data'):
    foldername = files.split('.')[0]
    os.system("mkdir -p video_data/" + foldername)
    os.system("unrar e video_data/"+ files + " video_data/"+foldername)

! rm video_data/*.rar



UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from test_train_splits.rar


Would you like to replace the existing file test_train_splits/brush_hair_test_split1.txt
  7194 bytes, modified on 2010-12-13 16:16
with a new one
  7194 bytes, modified on 2010-12-13 16:16

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit E

Extracting  test_train_splits/hit_test_split2.txt                         33%  OK 
Extracting  test_train_splits/hit_test_split3.txt                         34%  OK 
Extracting  test_train_splits/hug_test_split1.txt                         34%  OK 
Extracting  test_train_splits/hug_test_split2.txt                         35%  OK 
Extracting  test_train_splits/hug_test_split3.txt                         35%  OK 
Extracting  test_train_splits/jump_test_split1.txt                        36%  OK 
Extracting  test_train_splits/jump_test_split2.txt                        37%  OK 
Extract

In [8]:
import time
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
import torch
import transforms as T
from movinets import MoViNet
from movinets.config import _C

torch.manual_seed(97)
num_frames = 16 # 16
clip_steps = 2
Bs_Train = 16
Bs_Test = 16

transform = transforms.Compose([  
                                                     
                                 T.ToFloatTensorInZeroOne(),
                                 T.Resize((200, 200)),
                                 T.RandomHorizontalFlip(),
                                 #T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
                                 T.RandomCrop((172, 172))])
transform_test = transforms.Compose([                           
                                 T.ToFloatTensorInZeroOne(),
                                 T.Resize((200, 200)),
                                 #T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
                                 T.CenterCrop((172, 172))])

hmdb51_train = torchvision.datasets.HMDB51('video_data/', 'test_train_splits/', num_frames,frame_rate=5,
                                                step_between_clips = clip_steps, fold=1, train=True,
                                                transform=transform, num_workers=2)


hmdb51_test = torchvision.datasets.HMDB51('video_data/', 'test_train_splits/', num_frames,frame_rate=5,
                                                step_between_clips = clip_steps, fold=1, train=False,
                                                transform=transform_test, num_workers=2)
train_loader = DataLoader(hmdb51_train, batch_size=Bs_Train, shuffle=True)
test_loader  = DataLoader(hmdb51_test, batch_size=Bs_Test, shuffle=False)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/video_utils.py:215: UserWarning: There aren't enough frames in the current video to get a clip for the given clip length and frames between clips. The video (and potentially others) will be skipped.
  warnings.warn("There aren't enough frames in the current video to get a clip for the given clip length and "


In [9]:
def train_iter(model, optimz, data_load, loss_val):
    samples = len(data_load.dataset)
    model.train()
    model.cuda()
    model.clean_activation_buffers()
    optimz.zero_grad()
    for i, (data,_ , target) in enumerate(data_load):
        out = F.log_softmax(model(data.cuda()), dim=1)
        loss = F.nll_loss(out, target.cuda())
        loss.backward()
        optimz.step()
        optimz.zero_grad()
        model.clean_activation_buffers()
        if i % 50 == 0:
            print('[' +  '{:5}'.format(i * len(data)) + '/' + '{:5}'.format(samples) +
                  ' (' + '{:3.0f}'.format(100 * i / len(data_load)) + '%)]  Loss: ' +
                  '{:6.4f}'.format(loss.item()))
            loss_val.append(loss.item())
 
def evaluate(model, data_load, loss_val):
    model.eval()
    
    samples = len(data_load.dataset)
    csamp = 0
    tloss = 0
    model.clean_activation_buffers()
    with torch.no_grad():
        for data, _, target in data_load:
            output = F.log_softmax(model(data.cuda()), dim=1)
            loss = F.nll_loss(output, target.cuda(), reduction='sum')
            _, pred = torch.max(output, dim=1)
            
            tloss += loss.item()
            csamp += pred.eq(target.cuda()).sum()
            model.clean_activation_buffers()
    aloss = tloss / samples
    loss_val.append(aloss)
    print('\nAverage test loss: ' + '{:.4f}'.format(aloss) +
          '  Accuracy:' + '{:5}'.format(csamp) + '/' +
          '{:5}'.format(samples) + ' (' +
          '{:4.2f}'.format(100.0 * csamp / samples) + '%)\n')

In [10]:
N_EPOCHS = 1

model = MoViNet(_C.MODEL.MoViNetA0, 600,causal = False, pretrained = True, tf_like = True )
start_time = time.time()

trloss_val, tsloss_val = [], []
model.classifier[3] = torch.nn.Conv3d(2048, 51, (1,1,1))
optimz = optim.Adam(model.parameters(), lr=0.00005)
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    train_iter(model, optimz, train_loader, trloss_val)
    evaluate(model, test_loader, tsloss_val)
 
print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

Downloading: "https://github.com/Atze00/MoViNet-pytorch/blob/main/weights/modelA0_statedict_v2?raw=true" to /root/.cache/torch/hub/checkpoints/modelA0_statedict_v2



Epoch: 1
[    0/ 7519 (  0%)]  Loss: 3.9385
[  800/ 7519 ( 11%)]  Loss: 3.4501
[ 1600/ 7519 ( 21%)]  Loss: 3.0156
[ 2400/ 7519 ( 32%)]  Loss: 2.7325
[ 3200/ 7519 ( 43%)]  Loss: 2.3040
[ 4000/ 7519 ( 53%)]  Loss: 2.4667
[ 4800/ 7519 ( 64%)]  Loss: 1.2107
[ 5600/ 7519 ( 74%)]  Loss: 1.6366
[ 6400/ 7519 ( 85%)]  Loss: 1.8526
[ 7200/ 7519 ( 96%)]  Loss: 1.8699

Average test loss: 1.4780  Accuracy: 1886/ 2944 (64.06%)

Execution time: 1388.27 seconds
